In [1]:
// 先引入要使用的库
%use kandy
%use lib-ext
%use dataframe

根据 Fault Formation 算法,每次迭代在矩形内随机生成的一条直线，在直线右侧累加当前迭代的高度值。

比如迭代2次，随机生成恰好相交，矩形内就会出现4种不同的高度，如下图所示。

高度矩阵如下：

```txt
       0       0       0       0       0       0       0       0
      33       0       0       0       0       0       0       0
      33      33      33      33       0       0       0      66
      33      33      33      33      33      33     100      66
      33      33      33      33      33     100     100     100
      33      33      33      33     100     100     100     100
      33      33      33      33     100     100     100     100
      33      33      33     100     100     100     100     100
```

In [2]:
Image("./images/grid-8.png").withWidth(160)

随着迭代次数增加，可以预期随着直线不断生成，会出现某些位置频繁出现在直线右侧，使得该位置高度不断叠加，形成”高山“，某些位置频繁出现在直线左侧，形成“低谷”。

如下图，迭代16次就形成下面的不规则高度差。从 3D 图片更直观看出生成的 heightmap 比较接近大自然中高低起伏的山脉。

In [3]:
Image("./images/iteration-16.png").withWidth(512)

In [4]:
Image("./images/iteration-16-3d.png")

下面讨论一下迭代过程的高度计算。

根据每次迭代里计算高度的代码片段

```c
iHeight = iMaxDelta - ( ( iMaxDelta - iMinDelta) * iCurrentIteration ) / iIterations
```

可以得出`iHeight` 一个递减的序列，如下图红色点所示。

我们计算 heightmap 高度是将每次迭代的`iHeight`累加起来。请看书中C代码：

```c
fTempBuffer[(z * m_iSize) + x] += iHeight
```

累加起来的高度，开始会较快的变大，到后面变大的速度会变慢。如下图蓝色点所示。

反映到生成的 heightmap 就是，高度较高的区块高度差异不大，高度较低区块间高度差异大。

这是比较符合逻辑的，不太会存在一座较高的山比周围的山高出特别多。

In [5]:
val maxDelta = 255
val minDelta = 0;

val iterations = 32
val heights = mutableListOf<Float>()
val accumulateHeights = mutableListOf<Float>()
for (i in 0..iterations) {
    heights.add(
        maxDelta - ((maxDelta - minDelta) * i).toFloat() / iterations
    )
    accumulateHeights.add(heights.sum())
}
val index = (0..iterations).toList<Int>()

plot(mapOf(
    "index" to (0..iterations).toList<Int>(),
    "heights" to heights,
    "accHeights" to accumulateHeights
)) {
    points {
        x("index")
        y("heights")
        color = Color.RED
    }
       points {
        x("index")
        y("accHeights")
    }
}

生成 heightmap 后，下一步就是不同高度差之间过滤变得平滑。

书中使用 FIR filter 算法。

为了简化问题，我们只考虑一维。下面给出了实际生成 heightmap 的第一行数据。

In [6]:
val samples = listOf<Double>(
    47.0,47.0,47.0,47.0,47.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,48.0,48.0,
    58.0,58.0,58.0,58.0,58.0,48.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,53.0,
    53.0,42.0,42.0,42.0,42.0,42.0,55.0,55.0,55.0,63.0,63.0,63.0,63.0,63.0,62.0,
    49.0,58.0,61.0,61.0,61.0,61.0,61.0,61.0,61.0,61.0,61.0,61.0,61.0,61.0,50.0,
    50.0,50.0,50.0,47.0
)
val listSize = samples.size
val gIndex = (0 until listSize).toList<Int>();

plot(mapOf(
    "points" to samples,
    "index" to gIndex,
)) {
    points {
        x("index")
        y("points")
    }
    line {
        x("index")
        y("points")
    }
}

下面直接根据书中代码给 FIR filter 实现。

```kt
// give number a,b
b = a * weight + (1 - weight) * b
```

其它核心想法是：将前后两个值（a,b)按权重混合（mixin）得到 b 的新值。然后 b 再跟 c 按权重混合得到 c 的新值。

这个权重就是下面函数参数`filter`。

如果`filter = 0`，那就表示不做平滑处理；`filter = 1`，就会将所有值都改初始值。

In [7]:
fun firFilter(band: MutableList<Double>, direction: Int, filter: Float) {
    val start: Int = if (direction == 1) 0 else {band.size - 1}
    var value = band[start]
    
    // val j = stride
    var i = start
    repeat(band.size) {
        band[i] = filter * value + (1 - filter) * band[i]
        value = band[i]
        i += direction
    }
}

给出`[1, 1, 3, 3, 3]` 5个点，分别迭代 1次、2次、3次。

从下图可以看出，每次迭代都会实现曲线更加平滑。

迭代次数过多，会导致曲线退化成执行。

In [8]:
val lines: List<Double> = listOf(1.0, 1.0, 3.0, 3.0, 3.0)

val index3: List<Double> = (0 until lines.size).map{ it.toDouble() }.toList<Double>();


val iterations = listOf(0,1,2,4,8,16,32,64,128).map { i ->
    val iteration: MutableList<Double> = lines.toMutableList()
    repeat(i) {
        firFilter(iteration, 1, 0.3f)
    }
    
    "iteration $i" to iteration
}.toMap().toMutableMap()
val keys = iterations.keys.toList()

iterations["xIndex"] = index3.toMutableList()
val itetationResults = keys.map {key ->
    plot(iterations) {
        points {
            x("xIndex")
            y(key)
        }
        line {
            x("xIndex")
            y(key)
        }
        layout.title = key
    }
}

plotGrid(itetationResults, nCol = 3)

修改`filter` 也会影响曲线的平滑度,`filter`越大就会越平滑。

In [9]:
val filterResults = listOf(0.0f, 0.3f, 0.6f, 0.9f, 0.98f, 1.0f).map { filter ->
    val list = lines.toMutableList();
    firFilter(list, 1, filter)
    filter to list;
}.map { (filter, list) ->
    "filter ${filter}" to list
}.toMap().toMutableMap()

val keys = filterResults.keys.toList();
val colors = listOf(Color.BLACK, Color.BLUE, Color.GREEN, Color.PURPLE, Color.RED, Color.YELLOW)

filterResults["xIndex"] = index3.toMutableList();

val filterPlots = keys.mapIndexed {i, key ->
    plot(filterResults) {
        points {
            x("xIndex")
            y(key)
//             color = colors[i]
        }
        line {
            x("xIndex")
            y(key)
//             color = colors[i]
        }
         layout.title = key
    }

}

plotGrid(filterPlots, nCol = 3)

根据上面给出的公式，我们有以下推论：

1. 对于给定数组，重复执行`firFilter`将会得到更加平滑的结果。
2. `firFilter`执行跟方向有关， 为了得到平滑的结果从左到右和从右到左都要执行

下面给出四种情况的结果对比：

1. 从左到右执行 1 次
1. 从左到右执行 4 次
1. 从左到右和从右到左执行 1 次
1. 从左到右和从右到左执行 4 次

In [10]:
// left to right
val oneTimes = samples.toMutableList()
firFilter(oneTimes, 1, 0.6f)

// left to right 4 time
val fourTimes = samples.toMutableList()
repeat(4) {
    firFilter(fourTimes,1, 0.6f)
}

// left to right and right to left
var towDirection = samples.toMutableList()
firFilter(towDirection, 1, 0.6f)
firFilter(towDirection, -1, 0.6f)

// left to right and right to left 4 times
var towDirectionFourTime = samples.toMutableList()
repeat(4) {
    firFilter(towDirectionFourTime, 1, 0.6f)
    firFilter(towDirectionFourTime, -1, 0.6f)
}

val plotList = listOf(
    oneTimes to Color.BLUE,
    fourTimes to Color.GREEN,
    towDirection to Color.PURPLE,
    towDirectionFourTime to Color.RED,
).map {(list, c) ->
    plot(mapOf(
        "points" to list,
        "index" to gIndex,
    )) {
        line {
            x("index")
            y("points")
            color = c
        }
        points {
            x("index")
            y("points")
            color = c
        }
    }
}

plotGrid(plotList, nCol = 2)

”从左到右和从右到左执行 4 次“得到结果最好